In [1]:
# define rooms and items
game_room = {
    "name": "game room",
    "type": "room",
}
bedroom_1 = {
    "name": "bedroom_1",
    "type": "room",
}
bedroom_2 = {
    "name": "bedroom_2",
    "type": "room",
}
living_room = {
    "name": "living_room",
    "type": "room",
}
outside = {
    "name": "outside",
    "type": "room",
}
door_a = {
    "name": "door a",
    "type": "door",
}
door_b = {
    "name": "door b",
    "type": "door",
}
door_c = {
    "name": "door c",
    "type": "door",
}
door_d = {
    "name": "door d",
    "type": "door",
}
couch = {
    "name": "couch",
    "type": "furniture",
}
piano = {
    "name": "piano",
    "type": "furniture",
}
queen_bed = {
    "name": "queen_bed",
    "type": "furniture",
}
dresser = {
    "name": "dresser",
    "type": "furniture",
}
double_bed = {
    "name": "double_bed",
    "type": "furniture",
}
dining_table = {
    "name": "dining_table",
    "type": "furniture",
}
key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}
key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}
key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}
key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}
all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]
all_doors = [door_a, door_b, door_c, door_d]

# relacion de los objetos del juego

object_relations = {
    "door a": [game_room, bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "door c": [bedroom_1, living_room],
    "door d": [outside],
    "game room": [couch, piano, door_a],
    "bedroom_1": [queen_bed, door_c, door_b],
    "bedroom_2": [double_bed, dresser, door_b, door_c],
    "living_room": [dining_table, door_d],
    "outside": [door_d],
    "piano": [key_a],
    "queen_bed": [key_b],
    "double_bed": [key_c],
    "dresser": [key_d],
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.
INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}


In [2]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Comienza el juego
    """
    print("¡Comienza tu aventura levantándote del sofá en la Game Room! Encuentra la llave entre las notas del piano para desbloquear el siguiente nivel. ¡La emoción te espera! ¿Aceptas el desafío?")
    play_room(game_state["current_room"])
    
def reiniciar_juego():
    """
    Reinicia el juego
    """
    print("\nDebajo de la dining table ha aparecido un gnomo asustado se te ha lanzado encima y te ha golpeado la cabeza.")
    print("Esta sensación te suena familiar te sientes mareado pero Qué es esto otra vez no!!!!!????")
    global game_state  #para referirme a la misma variable, en este caso game_state
    game_state = INIT_GAME_STATE.copy()

def play_room(room):
    """
    Comprobamos que la sala en la que estamos en la target room, de ser asi el juego continua 
    si no le damos la opción de explorar la sala y ver los objetos para que pueda continuar el juegoPlay a room. 
    
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Felicidades superaste la aventura, eres todo un escapista!!!!!")
    else:
        print("Ahora mismo te encuentras en: " + room["name"])
        intended_action = input("Quieres explorar lo que hay en la sala o prefires buscar dentro de algun objeto ").strip()
        if intended_action == "explorar":
            explore_room(room)
            play_room(room)
        elif intended_action == "buscar":
            examine_item(input("¿Donde quieres buscar? ").strip())
        else:
            print("Has debido de equivocarte, ¿Qué quieres explorar o buscar?")
            play_room(room)
        linebreak()
        
def explore_room(room):
    """
    Sacamos una lista de objetos que puede explorar para buscar las llaves.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("Tras explorar la habitación:" + room["name"] + " .Has encontrado: " + ", ".join(items))


def get_next_room_of_door(door_name, current_room):
    """
    Gracias a object_relation lo que hacemos es mandar la habitación que no esta asignada como current_room
    """
    connected_rooms = object_relations[door_name]
    for room in connected_rooms:
        if not current_room == room:
            return room
        
def examine_item(item_name):
    """
    Buscar dentro de los objetos
    He metido una trampa al final del juego que si buscas en la mesa del salon sale un gnomo y te manda al sofa del inicio.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if item["name"] == item_name:
            output = "Buscando en: " + item_name + ". "

            if item["type"] == "door":
                have_key = any(key["target"] == item for key in game_state["keys_collected"])

                if have_key:
                    output += "Puerta desbloqueada!."
                    next_room = get_next_room_of_door(item_name, current_room)
                else:
                    output += "La puerta está bloqueada, debes encontrar primero la llave."
            else:
                if item["name"] in object_relations and len(object_relations[item["name"]]) > 0:
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "Has encontrado una " + item_found["name"] + "!"
                elif item_name.lower() == "dining_table":
                    reiniciar_juego() 
                else:
                        output += "No has encontrado nada aquí, busca en otro sitio."

            print(output)
            break

    if output is None:
        print("El objeto que quieres buscar no se encuentra en esta habitación. Te recomiendo explorar lo que hay primero.")

    if next_room and input("Exacto! Con la llave puedes abrir la puerta. ¿Te atreves a entrar? 'si' or 'no'").strip() == 'si':
        play_room(next_room)
    else:
        play_room(current_room)


In [3]:
game_state = INIT_GAME_STATE.copy()

start_game()

¡Comienza tu aventura levantándote del sofá en la Game Room! Encuentra la llave entre las notas del piano para desbloquear el siguiente nivel. ¡La emoción te espera! ¿Aceptas el desafío?
Ahora mismo te encuentras en: game room
Quieres explorar lo que hay en la sala o prefires buscar dentro de algun objeto explorar
Tras explorar la habitación:game room .Has encontrado: couch, piano, door a
Ahora mismo te encuentras en: game room
Quieres explorar lo que hay en la sala o prefires buscar dentro de algun objeto couch
Has debido de equivocarte, ¿Qué quieres explorar o buscar?
Ahora mismo te encuentras en: game room
Quieres explorar lo que hay en la sala o prefires buscar dentro de algun objeto bucar
Has debido de equivocarte, ¿Qué quieres explorar o buscar?
Ahora mismo te encuentras en: game room
Quieres explorar lo que hay en la sala o prefires buscar dentro de algun objeto buscar
¿Donde quieres buscar? piano
Buscando en: piano. Has encontrado una key for door a!
Ahora mismo te encuentras e